In [ ]:
import pycolmap
import os
import numpy as np

def read_calibration_matrix(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        K = []
        for line in lines:
            if line.startswith('K'):
                line = line.split('=')[1].strip().strip('[]')
                line = line.replace(';', '')  # Remove semicolons
                K.extend([float(x) for x in line.split()])
        # Handle multi-line matrix
        for line in lines:
            if not line.startswith('K') and line.strip():
                line = line.strip().strip('[]')
                line = line.replace(';', '')  # Remove semicolons
                K.extend([float(x) for x in line.split()])
    return np.array(K).reshape(3, 3)

#this is the path to the images
image_path = '/Users/bhargavsrisainama/Documents/personal/Research/SfMDataset/images'
#this is the database file created that will be used to store the features and matches
database_path = '/Users/bhargavsrisainama/Documents/personal/Research/SfMDataset/database.db' 
#this is the path where the sparse reconstruction will be stored
sparse_path = '/Users/bhargavsrisainama/Documents/personal/Research/sfm/sparse'
#this is the path to the calibration file generated from sfm
calibration_path = '/Users/bhargavsrisainama/Documents/personal/Research/SfMDataset/calibration.txt'

# Read the calibration matrix
K = read_calibration_matrix(calibration_path)
fx, fy = float(K[0, 0]), float(K[1, 1])
cx, cy = float(K[0, 2]), float(K[1, 2])

# Create ImageReaderOptions with camera parameters
reader_options = pycolmap.ImageReaderOptions()
reader_options.camera_model = 'PINHOLE'
reader_options.camera_params = f"{fx},{fy},{cx},{cy}"  # Convert to comma-separated string

# Feature extraction
pycolmap.extract_features(
    database_path=database_path,
    image_path=image_path,
    reader_options=reader_options
)

# Exhaustive matching
pycolmap.match_exhaustive(database_path=database_path)

# Create sparse reconstruction directory
os.makedirs(sparse_path, exist_ok=True)

# Run the incremental mapping
maps = pycolmap.incremental_mapping(
    database_path=database_path,
    image_path=image_path,
    output_path=sparse_path
)

# Save the reconstruction from the first model (folder `0`)
maps[0].write(os.path.join(sparse_path, '0'))

W20241111 13:54:53.230570 0x1f1def840 feature_extraction.cc:406] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20241111 13:54:53.230985 0x1717e3000 misc.cc:198] 
Feature extraction
I20241111 13:54:53.231415 0x171d5b000 sift.cc:722] Creating SIFT CPU feature extractor
I20241111 13:54:53.231424 0x171de7000 sift.cc:722] Creating SIFT CPU feature extractor
I20241111 13:54:53.231428 0x171e73000 sift.cc:722] Creating SIFT CPU feature extractor
I20241111 13:54:53.231433 0x171eff000 sift.cc:722] Creating SIFT CPU feature extractor
I20241111 13:54:53.231443 0x171f8b000 sift.cc:722] Creating SIFT CPU feature extractor
I20241111 13:54:53.231461 0x172017000 sift.cc: